In [1]:
import pandas as pd

In [2]:
!pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
from langchain_groq.chat_models import ChatGroq

In [4]:
from google.colab import userdata
import os
os.environ["GROQ_API_KEY"] = userdata.get('my_groq_key')

In [5]:
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [8]:
features = pd.read_csv('/content/features.txt', sep='\s+', header=None, names=['index', 'feature_name'])

features['feature_name'] = features['feature_name'].apply(lambda x: x if not features['feature_name'].duplicated(keep=False).any() else x + "_" + str(features[features['feature_name'] == x].index[0]))

In [9]:
activity_labels = pd.read_csv('/content/activity_labels.txt', sep='\s+', header=None, names=['label', 'activity'])

In [ ]:
df = pd.DataFrame(features)

In [ ]:
df.head(15)

,index,feature_name
0,1,tBodyAcc-mean()-X_0
1,2,tBodyAcc-mean()-Y_1
2,3,tBodyAcc-mean()-Z_2
3,4,tBodyAcc-std()-X_3
4,5,tBodyAcc-std()-Y_4
5,6,tBodyAcc-std()-Z_5
6,7,tBodyAcc-mad()-X_6
7,8,tBodyAcc-mad()-Y_7
8,9,tBodyAcc-mad()-Z_8
9,10,tBodyAcc-max()-X_9


In [ ]:
df = pd.DataFrame(activity_labels)
df

,label,activity
0,1,WALKING
1,2,WALKING_UPSTAIRS
2,3,WALKING_DOWNSTAIRS
3,4,SITTING
4,5,STANDING
5,6,LAYING


In [10]:
X_train = pd.read_csv('/content/X_train.txt', sep='\s+', header=None)
X_train.columns = features['feature_name']

In [11]:
y_train = pd.read_csv('/content/y_train.txt', sep='\s+', header=None, names=['activity'])

In [12]:
X_test = pd.read_csv('/content/X_test.txt', sep='\s+', header=None)
X_test.columns = features['feature_name']
y_test = pd.read_csv('/content/y_test.txt', sep='\s+', header=None, names=['activity'])

In [13]:
accelerometer_features = [feature for feature in features['feature_name'] if 'Acc' in feature]  # Filtering the accelerometer features

X_train_acc = X_train[accelerometer_features]
X_test_acc = X_test[accelerometer_features]

y_train['activity'] = y_train['activity'].map(activity_labels.set_index('label')['activity'])
y_test['activity'] = y_test['activity'].map(activity_labels.set_index('label')['activity'])

print("Preprocessed Training Data:")
print(X_train_acc.head())
print("\nTraining Activity Labels:")
print(y_train.head())


Preprocessed Training Data:
feature_name  tBodyAcc-mean()-X_0  tBodyAcc-mean()-Y_1  tBodyAcc-mean()-Z_2  \
0                        0.288585            -0.020294            -0.132905   
1                        0.278419            -0.016411            -0.123520   
2                        0.279653            -0.019467            -0.113462   
3                        0.279174            -0.026201            -0.123283   
4                        0.276629            -0.016570            -0.115362   

feature_name  tBodyAcc-std()-X_3  tBodyAcc-std()-Y_4  tBodyAcc-std()-Z_5  \
0                      -0.995279           -0.983111           -0.913526   
1                      -0.998245           -0.975300           -0.960322   
2                      -0.995380           -0.967187           -0.978944   
3                      -0.996091           -0.983403           -0.990675   
4                      -0.998139           -0.980817           -0.990482   

feature_name  tBodyAcc-mad()-X_6  tBodyA

In [ ]:
example_data = X_train_acc.iloc[0]

data_str = example_data.to_string(index=False)

def create_custom_query(data_str):
    query = f'''
    You are a machine learning model expert.
    Your task is to analyze the given featurized accelerometer data and predict the human activity.
    The data is provided below in a featurized format.
    Provide the predicted activity label and, if necessary, a brief explanation of your reasoning.

    Featurized Data:
    {data_str}
    '''
    return query

query = create_custom_query(data_str)
model_name = 'llama3-70b'
llm = ChatGroq(model=groq_models[model_name], api_key=os.environ["GROQ_API_KEY"], temperature=0.5)
answer = llm.invoke(query)

print(answer.content)


After analyzing the featurized accelerometer data, I predict that the human activity is **Walking**.

Here's my reasoning:

1. **Frequency domain features**: The data contains many high-frequency components, which are typical of walking activities. Walking involves rapid movements of the legs, arms, and body, resulting in high-frequency oscillations in the accelerometer data.
2. **Magnitude and variance**: The data exhibits a moderate to high magnitude and variance, indicating that the activity involves significant movements and changes in acceleration. Walking is a dynamic activity that involves periodic movements of the legs, which would result in moderate to high acceleration values.
3. **Correlation and spectral features**: The data shows a strong correlation between the x, y, and z axes, which is typical of walking activities. The spectral features also indicate a strong presence of frequency components in the 1-5 Hz range, which is consistent with the frequency range of human wal

In [ ]:
example_data

,0
feature_name,
tBodyAcc-mean()-X_0,0.288585
tBodyAcc-mean()-Y_1,-0.020294
tBodyAcc-mean()-Z_2,-0.132905
tBodyAcc-std()-X_3,-0.995279
tBodyAcc-std()-Y_4,-0.983111
...,...
fBodyBodyAccJerkMag-meanFreq()_525,0.346989
fBodyBodyAccJerkMag-skewness()_526,-0.516080
fBodyBodyAccJerkMag-kurtosis()_527,-0.802760


In [ ]:
y_train.head()

,activity
0,STANDING
1,STANDING
2,STANDING
3,STANDING
4,STANDING


In [ ]:
X_train.iloc[0]

,0
feature_name,
tBodyAcc-mean()-X_0,0.288585
tBodyAcc-mean()-Y_1,-0.020294
tBodyAcc-mean()-Z_2,-0.132905
tBodyAcc-std()-X_3,-0.995279
tBodyAcc-std()-Y_4,-0.983111
...,...
"angle(tBodyGyroMean,gravityMean)_556",-0.464761
"angle(tBodyGyroJerkMean,gravityMean)_557",-0.018446
"angle(X,gravityMean)_558",-0.841247


# **Q1: Demonstrate how to use Zero-Shot Learning and Few-Shot Learning to classify human activities based on the featurized accelerometer data. Qualitatively demonstrate the performance of Few-Shot Learning with Zero-Shot Learning. Which method performs better? Why? [1 marks]**

In [ ]:
# Zero-Shot learning

sample_data = X_train_acc.iloc[0]                              # Passing the 1st activity from the train set.

data_str = sample_data.to_string(index=False)                  # LLMs can understand when we pass them in text(string) than in numbers

def create_query(data_str):
    query = f'''
    You are a machine learning model expert.
    Your task is to analyze the given featurized accelerometer data and predict the human activity.
    The data is provided below in a featurized format.
    Provide the predicted activity label and, if necessary, a brief explanation of your reasoning.

    Featurized Data:
    {data_str}
    '''
    return query                                               # This returns the query

query = create_query(data_str)
model_name = 'llama3-70b'
llm = ChatGroq(model=groq_models[model_name], api_key=os.environ["GROQ_API_KEY"], temperature=0)   #The LLM pipeline here.
answer = llm.invoke(query)

print(answer.content)                     # Prints the LLM's response

print('\nThe actual label of this activity performed is: ',y_train.iloc[0]['activity'])


After analyzing the featurized accelerometer data, I predict that the human activity is **Walking**.

Here's a brief explanation of my reasoning:

1. **Frequency domain features**: The data contains many frequency domain features, which are commonly used in human activity recognition tasks. The presence of these features suggests that the data is related to movement or activity recognition.
2. **High-energy patterns**: The data exhibits high-energy patterns, which are characteristic of activities that involve rapid movements, such as walking or running.
3. **Periodic patterns**: The data also shows periodic patterns, which are typical of activities that involve repetitive movements, such as walking or cycling.
4. **Low-frequency components**: The presence of low-frequency components in the data suggests that the activity involves movements that are not too rapid or jerky, which is consistent with walking.
5. **Lack of high-amplitude spikes**: The data does not contain high-amplitude sp

In [ ]:
#Few shot learning

sample_data = X_train_acc.iloc[2]                              # Passing the 1st activity from the train set.

data_str = sample_data.to_string(index=False)                  # LLMs can understand when we pass them in text(string) than in numbers

def create_query(data_str):
    query = f'''
    You are a machine learning model expert.
    Your task is to analyze the given featurized accelerometer data and predict the human activity.
    The data is provided below in a featurized format.
    Provide the predicted activity label and, if necessary, a brief explanation of your reasoning.

    Few examples are provided for your reference:
    Few-Shot Examples:
      Example 1:
      Featurized Data:
      - X-axis: Moderate oscillations between -0.2 to 0.2
      - Y-axis: Moderate oscillations between -0.3 to 0.3
      - Z-axis: Moderate oscillations between -0.4 to 0.4
      Activity: WALKING

      Example 2:
      Featurized Data:
      - X-axis: Higher oscillations between -0.5 to 0.5
      - Y-axis: Moderate oscillations between -0.2 to 0.4
        - Z-axis: Higher oscillations between -0.6 to 0.6
        Activity: WALKING_UPSTAIRS

      Example 3:
      Featurized Data:
      - X-axis: Moderate oscillations between -0.3 to 0.3
      - Y-axis: Higher variability between -0.4 to 0.6
      - Z-axis: Higher variability between -0.5 to 0.5
      Activity: WALKING_DOWNSTAIRS

      Example 4:
      Featurized Data:
      - X-axis: Low values between -0.1 to 0.1
      - Y-axis: Low values between -0.1 to 0.1
      - Z-axis: Low values between -0.1 to 0.1
      Activity: SITTING

      Example 5:
      Featurized Data:
      - X-axis: Low values between -0.1 to 0.1
      - Y-axis: Slightly higher but stable values between -0.2 to 0.2
      - Z-axis: Low values between -0.1 to 0.1
      Activity: STANDING

      Example 6:
      Featurized Data:
      - X-axis: Very low values close to 0, between -0.05 to 0.05
      - Y-axis: Very low values close to 0, between -0.05 to 0.05
      - Z-axis: Very low values close to 0, between -0.05 to 0.05
      Activity: LAYING

      Featurized Data:
      {data_str}
    '''
    return query                                               # This returns the query

query = create_query(data_str)
model_name = 'llama3-70b'
llm = ChatGroq(model=groq_models[model_name], api_key=os.environ["GROQ_API_KEY"], temperature=0)   #The LLM pipeline here.
answer = llm.invoke(query)

print(answer.content)                     # Prints the LLM's response

print('\nThe actual label of this activity performed is: ',y_train.iloc[0]['activity'])

After analyzing the provided featurized accelerometer data, I predict that the human activity is **STANDING**.

Here's my reasoning:

* The values in the data are mostly low to moderate, with a range of -1 to 1, which suggests that the activity is not very intense or dynamic.
* The X-axis values are relatively stable, with a small range of -0.3 to 0.3, indicating minimal movement in the horizontal direction.
* The Y-axis values are slightly higher, with a range of -0.4 to 0.4, suggesting some movement in the vertical direction, but still relatively stable.
* The Z-axis values are also relatively stable, with a range of -0.5 to 0.5, indicating minimal movement in the vertical direction.

Comparing these patterns to the provided examples, I notice that they are most similar to the patterns observed in the **STANDING** activity (Example 5). The values are not as low as those in **SITTING** or **LAYING**, and not as high as those in **WALKING** or **WALKING_UPSTAIRS**. The stability of the

**The above examples shows that using Few-shot learning is more beneficial than zero-shot learnin, because the model atleast knows somewhat which range of values correspond to which activity. Increasing the utility of few-shot is still under our hands, since we are the one providing the "few" labelled examples for it.**

# **Q2: Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better? Why? [1 marks]**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

subset_size = 500
X_test_subset = X_test_acc[:subset_size]
y_test_subset = y_test['activity'][:subset_size]

# Train a Decision Tree Classifier
decision_tree = DecisionTreeClassifier(random_state=101)
decision_tree.fit(X_train_acc, y_train['activity'])

# Predict the activities on the test subset
y_pred = decision_tree.predict(X_test_subset)

# Calculate accuracy for the Decision Tree model
accuracy = accuracy_score(y_test_subset, y_pred)
print(f"Decision Tree Accuracy on Test Subset: {accuracy}")




Decision Tree Accuracy on Test Subset: 0.768


In [16]:
from sklearn.metrics import accuracy_score
subset_size = 5
X_test_subset = X_test_acc[:subset_size]
y_test_subset = y_test['activity'][:subset_size]

def get_llm_predictions(X_test_subset):
    predictions = []
    for i, row in X_test_subset.iterrows():
        query = f'''
        You are a machine learning model expert.
        Your task is to analyze the given featurized accelerometer data and predict the human activity.
        Directly give the predicted activity in single word with no explanation.

        Example 1:
        Featurized Data:
        - X-axis: Moderate oscillations between -0.2 to 0.2
        - Y-axis: Moderate oscillations between -0.3 to 0.3
        - Z-axis: Moderate oscillations between -0.4 to 0.4
        Activity: WALKING

        Example 2:
        Featurized Data:
        - X-axis: Higher oscillations between -0.5 to 0.5
        - Y-axis: Moderate oscillations between -0.2 to 0.4
        - Z-axis: Higher oscillations between -0.6 to 0.6
        Activity: WALKING_UPSTAIRS

        Example 3:
        Featurized Data:
        - X-axis: Moderate oscillations between -0.3 to 0.3
        - Y-axis: Higher variability between -0.4 to 0.6
        - Z-axis: Higher variability between -0.5 to 0.5
        Activity: WALKING_DOWNSTAIRS

        Example 4:
        Featurized Data:
        - X-axis: Low values between -0.1 to 0.1
        - Y-axis: Low values between -0.1 to 0.1
        - Z-axis: Low values between -0.1 to 0.1
        Activity: SITTING

        Example 5:
        Featurized Data:
        - X-axis: Low values between -0.1 to 0.1
        - Y-axis: Slightly higher but stable values between -0.2 to 0.2
        - Z-axis: Low values between -0.1 to 0.1
        Activity: STANDING

        Example 6:
        Featurized Data:
        - X-axis: Very low values close to 0, between -0.05 to 0.05
        - Y-axis: Very low values close to 0, between -0.05 to 0.05
        - Z-axis: Very low values close to 0, between -0.05 to 0.05
        Activity: LAYING

        Predict the activity for the following featurized data:
        Featurized Data: {row.to_list()}
        Activity: ?
        '''

        model_name = 'llama3-70b'
        llm = ChatGroq(model=groq_models[model_name], api_key=os.environ["GROQ_API_KEY"], temperature=0)   #The LLM pipeline here.
        answer = llm.invoke(query)
        predicted_activity = answer.content.strip()  # Assuming LLM returns the activity label directly

        predictions.append(predicted_activity)

    return predictions

# Get LLM predictions on the test subset
y_pred_llm = get_llm_predictions(X_test_subset)

# Calculate accuracy for the LLM Few-Shot Learning
accuracy_llm = accuracy_score(y_test_subset, y_pred_llm)
print(f"LLM Few-Shot Learning Accuracy on Test Subset: {accuracy_llm:.4f}")

LLM Few-Shot Learning Accuracy on Test Subset: 0.0000


# **Q3: What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data? [1 marks]**

**Limitations in using Zero-shot learning:**

-> Relies heavily on external knowledge bases or semantic embeddings, making it a must for them to be comprehensive and accurate.

-> Incomplete or biased external knowledge can disrupt the model's ability to make accurate predictions for unseen classes.

-> Evaluating zero-shot learning models can be difficult, as it requires robust benchmarks that accurately reflect real-world scenarios with unseen classes.

->The complexity of semantic relationships and the need for detailed attribute descriptions can limit the model's scalability and performance.

**Limitations in using Few-shot learning:**

-> The quality and representativeness of the few available examples are crucial. Poor-quality or non-representative examples can significantly degrade model performance.

-> Few-shot learning heavily relies on the assumption that the few provided examples are sufficient to capture the variability of the class, which is not always the case.

-> With limited training examples, there's a high risk of overfitting to the few examples provided.



# Q4: **What does the model classify when given input from an entirely new activity that it hasn't seen before? [0.5 mark]**

In [ ]:
# Zero-Shot learning

sample_data = X_test_acc.iloc[10]                              # Passing the 1st activity from the train set.

data_str = sample_data.to_string(index=False)                  # LLMs can understand when we pass them in text(string) than in numbers

def create_query(data_str):
    query = f'''
    You are a machine learning model expert.
    Your task is to analyze the given featurized accelerometer data and predict the human activity.
    The data is provided below in a featurized format.
    Provide the predicted activity label and, if necessary, a brief explanation of your reasoning.

    The 6 labels of class are: 1 WALKING
                               2 WALKING_UPSTAIRS
                               3 WALKING_DOWNSTAIRS
                               4 SITTING
                               5 STANDING
                               6 LAYING


    Featurized Data:
    {data_str}
    '''
    return query                                               # This returns the query

query = create_query(data_str)
model_name = 'llama3-70b'
llm = ChatGroq(model=groq_models[model_name], api_key=os.environ["GROQ_API_KEY"], temperature=0)   #The LLM pipeline here.
answer = llm.invoke(query)

print(answer.content)                     # Prints the LLM's response

print('\nThe actual label of this activity performed is: ',y_test.iloc[10]['activity'])

After analyzing the featurized accelerometer data, I predict that the human activity is **STANDING** (label 5).

Here's a brief explanation of my reasoning:

1. **Low variability**: The data exhibits low variability, which is characteristic of standing or sitting activities. If the person were walking or engaging in more dynamic activities, the accelerometer readings would show more fluctuations.
2. **Low amplitude**: The amplitude of the accelerometer readings is relatively low, which suggests that the person is not moving vigorously. This is consistent with standing or sitting activities.
3. **Periodic patterns**: I noticed some periodic patterns in the data, which could be indicative of the person's natural sway or slight movements while standing.
4. **Lack of prominent peaks**: There are no prominent peaks in the data, which would be expected if the person were walking, running, or engaging in other high-intensity activities.

While the data is not conclusive, and there might be so

# **Q5: Test the model with random data (ensuring the data has the same dimensions and range as the previous input) and report the results. [0.5 mark]**

In [ ]:
df = pd.read_csv('/content/linear_acc.csv')

In [ ]:
df.head()

,time,ax (m/s^2),ay (m/s^2),az (m/s^2),aT (m/s^2)
0,4.007,0.6597,-0.4309,0.4370,0.9010
1,4.027,0.7758,-1.7611,-2.1047,2.8520
2,4.047,1.3243,-3.2582,-3.7324,5.1285
3,4.067,1.1180,-2.5832,-3.5800,4.5554
4,4.087,0.9662,-1.6768,-3.2768,3.8061


In [ ]:
example_data = df.iloc[10]

data_str = example_data.to_string(index=False)

def create_query(data_str):

  query = f'''
    You are a machine learning model expert.
    Your task is to analyze the given featurized accelerometer data and predict the human activity.
    The data is provided below in a featurized format.
    Provide the predicted activity label and, if necessary, a brief explanation of your reasoning.
    1st feature: Time, 2nd feature: X-axis of acceleration, 3rd feature: Y-axis of acceleration,,
    4th feature: Z-axis of acceleration, 5th feature: Square root of sum of squares of x,y,z values.

    Featurized Data:
    {data_str}
    '''
  return query

query = create_query(data_str)
model_name = 'llama3-70b'
llm = ChatGroq(model=groq_models[model_name], api_key=os.environ["GROQ_API_KEY"], temperature=0)   #The LLM pipeline here.
answer = llm.invoke(query)

print(answer.content)


A featurized accelerometer dataset! Let's dive into it.

After analyzing the provided featurized data, I'll make a prediction about the human activity.

Here's the data:

1. Time: 4.2070 (not very informative for this prediction, but it's a timestamp)
2. X-axis acceleration: 2.6105
3. Y-axis acceleration: 0.0327
4. Z-axis acceleration: -0.5270
5. Square root of sum of squares of x, y, z values: 2.6950

Based on the acceleration values, I notice that:

* The X-axis acceleration is relatively high (2.6105), indicating a significant movement in the horizontal direction.
* The Y-axis acceleration is very low (0.0327), suggesting minimal movement in the vertical direction.
* The Z-axis acceleration is negative (-0.5270), which might indicate a slight downward movement.

The square root of the sum of squares of x, y, z values (2.6950) is a measure of the overall acceleration magnitude. In this case, it's moderate.

Considering these factors, I'm going to predict that the human activity is:



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install git -y
!apt-get install gh -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  gh
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 6,242 kB of archives.
After this operation, 33.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 gh amd64 2.4.0+dfsg1-2 [6,242 kB]
Fetched 6,242 kB in 1s (6,714 kB/s)
Selecting previously unselected package gh.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../gh_2.4.0+dfsg1-2_amd64.deb ...
Unpacking gh (2.4.0+dfsg1-2) ...
Setting up gh (2.4.0+dfsg1-2) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
!gh repo clone Suruchi-Hardaha/https://github.com/Suruchi-Hardaha/es335-24-fall-assignment-1


Welcome to GitHub CLI!

To authenticate, please run `gh auth login`.
